In [1]:
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import missingno as msno
import plotly.graph_objects as go
from datetime import datetime
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import f_classif, mutual_info_classif, VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE
np.random.seed(42)


#to ignore warnings
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Allow long lines

In [2]:
# Specify the path to the CSV file in the parent directory
file_path = "../Dataset/pd_speech_features/pd_speech_features.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, sep=',',skiprows=1)

In [3]:
df.head()

id  gender      PPE      DFA     RPDE  numPulses  numPeriodsPulses  \
0   0       1  0.85247  0.71826  0.57227        240               239   
1   0       1  0.76686  0.69481  0.53966        234               233   
2   0       1  0.85083  0.67604  0.58982        232               231   
3   1       0  0.41121  0.79672  0.59257        178               177   
4   1       0  0.32790  0.79782  0.53028        236               235   

   meanPeriodPulses  stdDevPeriodPulses  locPctJitter  locAbsJitter  \
0          0.008064            0.000087       0.00218      0.000018   
1          0.008258            0.000073       0.00195      0.000016   
2          0.008340            0.000060       0.00176      0.000015   
3          0.010858            0.000183       0.00419      0.000046   
4          0.008162            0.002669       0.00535      0.000044   

   rapJitter  ppq5Jitter  ddpJitter  locShimmer  locDbShimmer  apq3Shimmer  \
0    0.00067     0.00129    0.00200     0.05883         0.517      0.03011   
1    0.00052     0.00112    0.00157     0.05516         0.502      0.02320   
2    0.00057     0.00111    0.00171     0.09902         0.897      0.05094   
3    0.00149     0.00268    0.00446     0.05451         0.527      0.02395   
4    0.00166     0.00227    0.00499     0.05610         0.497      0.02909   

   apq5Shimmer  apq11Shimmer  ddaShimmer  meanAutoCorrHarmonicity  \
0      0.03496       0.04828     0.09034                 0.970805   
1      0.03675       0.06195     0.06961                 0.984322   
2      0.06497       0.07772     0.15282                 0.974846   
3      0.02857       0.04462     0.07185                 0.968343   
4      0.03327       0.05278     0.08728                 0.975754   

   meanNoiseToHarmHarmonicity  meanHarmToNoiseHarmonicity  minIntensity  \
0                    0.036223                      18.995     69.997496   
1                    0.017974                      21.497     67.415903   
2                    0.026313                      17.651     62.661706   
3                    0.042003                      19.865     76.306989   
4                    0.027139                      19.557     76.645686   

   maxIntensity  meanIntensity          f1           f2           f3  \
0     76.088046      72.465512  539.342735  1031.849040  2447.162183   
1     73.046374      71.528945  564.363615  1016.367294  2383.565201   
2     71.633549      68.086583  548.444604  1032.406341  2357.826954   
3     81.000749      79.190593  819.529588  1201.813897  3154.035654   
4     80.937258      79.183495  846.796144  1215.346469  3201.513132   

            f4          b1          b2          b3          b4  GQ_prc5_95  \
0  3655.054806  101.092218   83.147440  255.214830  396.643631     0.77778   
1  3498.681572   58.465428   86.487292  248.357127  218.229722     0.81250   
2  3678.128717  160.387771   54.685168  151.694847   84.240339     0.81818   
3  4122.163933  238.667052  191.984916  573.752909  526.147599     0.98548   
4  4085.456839  402.216738  210.061394  203.637106  384.611697     0.97847   

   GQ_std_cycle_open  GQ_std_cycle_closed  GNE_mean  GNE_std  GNE_SNR_TKEO  \
0            11.7245               2.8277   1.17300  0.26512      0.083127   
1            13.8284               2.8908   1.02210  0.22004      0.127410   
2            26.9273               2.6975   0.84951  0.15756      0.116890   
3           139.5744               1.6961   0.83405  0.17295      0.147370   
4           102.0549              15.4045   0.83556  0.16210      0.151990   

   GNE_SNR_SEO  GNE_NSR_TKEO  GNE_NSR_SEO  VFER_mean  VFER_std  VFER_entropy  \
0  1200445.612        1.5347       3.0152   0.000463  0.000260       0.83621   
1  1298455.445        1.6029       3.0600   0.000615  0.000295       1.04550   
2  1272869.841        1.6223       3.0309   0.000360  0.000266       0.63668   
3  1932289.206        1.6717       3.0293   0.000196  0.000194       0.45452   
4  1861807.802        1.6781   

# Filtrar o Dataset para apenas apresentar dados do Sexo Feminino e sexo Masculino

In [4]:
female_df = df[df['gender'] == 0]
male_df = df[df['gender'] == 1]
female_df.shape
male_df.shape

(390, 755)

# Drop da Coluna de Id

In [5]:
female_df = female_df.drop("id", axis=1)
male_df = male_df.drop("id", axis=1)

# Divisão entre features e label

In [6]:
X_female = female_df.drop('class', axis=1)  # Features
y_female = female_df['class']  # Target variable
X_male = male_df.drop('class', axis=1)  # Features
y_male = male_df['class']  # Target variable

# SMOTE

In [7]:
smote_female = SMOTE(sampling_strategy='auto', random_state=42)
X_female_smote, y_female_smote = smote_female.fit_resample(X_female, y_female)

smote_male = SMOTE(sampling_strategy='auto', random_state=42)
X_male_smote, y_male_smote = smote_male.fit_resample(X_male, y_male)

In [8]:
unique, counts = np.unique(y_female, return_counts=True)
original_class_distribution = dict(zip(unique, counts))
unique, counts = np.unique(y_female_smote, return_counts=True)
resampled_class_distribution = dict(zip(unique, counts))

print("Original class distribution:", original_class_distribution)
print("Resampled class distribution:", resampled_class_distribution)

Original class distribution: {0: 123, 1: 243}
Resampled class distribution: {0: 243, 1: 243}


In [9]:
unique, counts = np.unique(y_male, return_counts=True)
original_class_distribution = dict(zip(unique, counts))
unique, counts = np.unique(y_male_smote, return_counts=True)
resampled_class_distribution = dict(zip(unique, counts))

print("Original class distribution:", original_class_distribution)
print("Resampled class distribution:", resampled_class_distribution)

Original class distribution: {0: 69, 1: 321}
Resampled class distribution: {0: 321, 1: 321}


# Normalização dos dois Datasets (Feminino e Masculino)

In [10]:
features = X_female_smote.columns

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Scale the features
X_female_smote_scaled = pd.DataFrame(scaler.fit_transform(X_female_smote[features]), columns=features)

In [11]:
features = X_male_smote.columns

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Scale the features
X_male_smote_scaled = pd.DataFrame(scaler.fit_transform(X_male_smote[features]), columns=features)

# Correlation

In [12]:
# with the following function we can select highly correlated features
# it will remove the first feature that is correlated with anything else
# without any further insight.

def correlation(dataset, threshold):
    
    # create a set where I will store the names of correlated columns
    col_corr = set()
    
    # create the correlation matrix
    corr_matrix = dataset.corr()
    
    # for each feature in the dataset (columns of the correlation matrix)
    for i in range(len(corr_matrix.columns)):
        
        # check with other features
        for j in range(i):
            
            # if the correlation is higher than a certain threshold
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                
                # print correlation, and variables examined
                # keep in mind that the columns and rows of the dataframe are identical
                # so we can identify the features being examned by looking for i,j
                # in the column names
                print(abs(corr_matrix.iloc[i, j]), corr_matrix.columns[i], corr_matrix.columns[j])
                
                # get the name of the correlated feature
                colname = corr_matrix.columns[j]
                
                # and add it to our correlated set
                col_corr.add(colname)
                
    return col_corr

In [13]:
corr_features_female = correlation(X_female_smote_scaled, 0.8)
corr_features_male = correlation(X_male_smote_scaled, 0.8)
print(len(set(corr_features_female)))
print(len(set(corr_features_male)))

0.9999460815644424 numPeriodsPulses numPulses
0.9027891354450992 meanPeriodPulses numPulses
0.9026458394763934 meanPeriodPulses numPeriodsPulses
0.9561477902685724 locAbsJitter locPctJitter
0.9587158530168505 rapJitter locPctJitter
0.9625658640253487 rapJitter locAbsJitter
0.9660429384212299 ppq5Jitter locPctJitter
0.951454143205803 ppq5Jitter locAbsJitter
0.9699975826011277 ppq5Jitter rapJitter
0.9587320691814611 ddpJitter locPctJitter
0.9626363370250992 ddpJitter locAbsJitter
0.9999941872028469 ddpJitter rapJitter
0.9700416050994086 ddpJitter ppq5Jitter
0.9959380227759249 locDbShimmer locShimmer
0.982907590008753 apq3Shimmer locShimmer
0.9767030438081709 apq3Shimmer locDbShimmer
0.9841284841711554 apq5Shimmer locShimmer
0.9773652427312904 apq5Shimmer locDbShimmer
0.9675872869069989 apq5Shimmer apq3Shimmer
0.9145265491337877 apq11Shimmer locShimmer
0.9156226070834663 apq11Shimmer locDbShimmer
0.8663046155327179 apq11Shimmer apq3Shimmer
0.9285285880029288 apq11Shimmer apq5Shimmer
0.982

0.9925947619406267 det_TKEO_std_7_coef det_entropy_shannon_7_coef
0.9889822076281258 det_TKEO_std_7_coef det_TKEO_mean_7_coef
0.8511252879634482 det_TKEO_std_8_coef Ed_8_coef
0.998458889442687 det_TKEO_std_8_coef det_entropy_shannon_8_coef
0.8616601367385709 det_TKEO_std_8_coef det_entropy_shannon_10_coef
0.9940135404186674 det_TKEO_std_8_coef det_TKEO_mean_8_coef
0.8594836421725714 det_TKEO_std_8_coef det_TKEO_mean_10_coef
0.8725394716383849 det_TKEO_std_9_coef Ea
0.8445918512163825 det_TKEO_std_9_coef Ed_8_coef
0.8781933849585837 det_TKEO_std_9_coef Ed_10_coef
0.9818928099562837 det_TKEO_std_9_coef det_entropy_shannon_9_coef
0.9866058055792399 det_TKEO_std_9_coef det_TKEO_mean_9_coef
0.8454362121030765 det_TKEO_std_10_coef Ed_10_coef
0.8782504653838792 det_TKEO_std_10_coef det_entropy_shannon_8_coef
0.9970063479481004 det_TKEO_std_10_coef det_entropy_shannon_10_coef
0.828593784758395 det_TKEO_std_10_coef det_TKEO_mean_8_coef
0.9981260307619123 det_TKEO_std_10_coef det_TKEO_mean_10_co

0.9970691192477354 app_TKEO_std_7_coef app_entropy_shannon_4_coef
0.9967913292463256 app_TKEO_std_7_coef app_entropy_shannon_5_coef
0.9961340098573205 app_TKEO_std_7_coef app_entropy_shannon_6_coef
0.9957391586039327 app_TKEO_std_7_coef app_entropy_shannon_7_coef
0.9951187303321394 app_TKEO_std_7_coef app_entropy_shannon_8_coef
0.9950921901271352 app_TKEO_std_7_coef app_entropy_shannon_9_coef
0.995017430669736 app_TKEO_std_7_coef app_entropy_shannon_10_coef
0.8150101097214749 app_TKEO_std_7_coef app_entropy_log_1_coef
0.8289095897746935 app_TKEO_std_7_coef app_entropy_log_2_coef
0.8505449344854522 app_TKEO_std_7_coef app_entropy_log_3_coef
0.8867663876599845 app_TKEO_std_7_coef app_entropy_log_4_coef
0.9028132495589584 app_TKEO_std_7_coef app_entropy_log_5_coef
0.911283443304846 app_TKEO_std_7_coef app_entropy_log_6_coef
0.9071879507809185 app_TKEO_std_7_coef app_entropy_log_7_coef
0.9105456455841836 app_TKEO_std_7_coef app_entropy_log_8_coef
0.9107989284315844 app_TKEO_std_7_coef app_

0.8283718209121991 det_LT_TKEO_mean_1_coef Ea
0.8736393029930584 det_LT_TKEO_mean_1_coef Ed_1_coef
0.853804672287016 det_LT_TKEO_mean_1_coef Ed_2_coef
0.9066402053028899 det_LT_TKEO_mean_1_coef Ed_3_coef
0.9078329802569196 det_LT_TKEO_mean_1_coef Ed_4_coef
0.876873333422754 det_LT_TKEO_mean_1_coef Ed_5_coef
0.9269833691615431 det_LT_TKEO_mean_1_coef det_entropy_shannon_1_coef
0.9250809691057326 det_LT_TKEO_mean_1_coef det_entropy_shannon_2_coef
0.8004890569579008 det_LT_TKEO_mean_1_coef det_entropy_shannon_3_coef
0.8614564862884555 det_LT_TKEO_mean_1_coef det_entropy_shannon_4_coef
0.938534377676289 det_LT_TKEO_mean_1_coef det_TKEO_mean_1_coef
0.9003434467330604 det_LT_TKEO_mean_1_coef det_TKEO_mean_2_coef
0.8523786047119617 det_LT_TKEO_mean_1_coef det_TKEO_std_1_coef
0.8584915539140551 det_LT_TKEO_mean_1_coef det_TKEO_std_2_coef
0.8446623310929464 det_LT_TKEO_mean_1_coef det_TKEO_std_4_coef
0.9937311810451749 det_LT_TKEO_mean_1_coef Ed2_1_coef
0.960620454367375 det_LT_TKEO_mean_1_coef

0.9447947117643511 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_1_coef
0.9561841852881278 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_2_coef
0.9707118783185331 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_3_coef
0.9862618939896223 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_4_coef
0.9949469892739548 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_5_coef
0.9989297499893958 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_6_coef
0.9991943546934794 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_7_coef
0.9999982393087243 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_8_coef
0.9999991184626581 app_LT_entropy_shannon_10_coef app_LT_entropy_shannon_9_coef
0.9569664235679842 app_LT_entropy_log_1_coef app_entropy_log_1_coef
0.9489341363705647 app_LT_entropy_log_1_coef app_entropy_log_2_coef
0.9302612268904528 app_LT_entropy_log_1_coef app_entropy_log_3_coef
0.8769321623746854 app_LT_entropy_log_1_coef app_entropy_log_4_coef
0.822975

...

0.878315952463194 tqwt_stdValue_dec_14 tqwt_TKEO_std_dec_14
0.8485114199263338 tqwt_stdValue_dec_14 tqwt_TKEO_std_dec_15
0.9189271124994951 tqwt_stdValue_dec_14 tqwt_stdValue_dec_13
0.8744557859724759 tqwt_stdValue_dec_15 tqwt_entropy_shannon_dec_14
0.9865144953400015 tqwt_stdValue_dec_15 tqwt_entropy_shannon_dec_15
0.8598544856062796 tqwt_stdValue_dec_15 tqwt_entropy_shannon_dec_16
0.8255622777296674 tqwt_stdValue_dec_15 tqwt_entropy_log_dec_14
0.8570025254157142 tqwt_stdValue_dec_15 tqwt_entropy_log_dec_15
0.9078132485590099 tqwt_stdValue_dec_15 tqwt_TKEO_mean_dec_15
0.8002348119246484 tqwt_stdValue_dec_15 tqwt_TKEO_mean_dec_16
0.9107359935075914 tqwt_stdValue_dec_15 tqwt_TKEO_std_dec_15
0.8873426905735252 tqwt_stdValue_dec_15 tqwt_TKEO_std_dec_16
0.9041505494250657 tqwt_stdValue_dec_15 tqwt_stdValue_dec_14
0.867941337839582 tqwt_stdValue_dec_16 tqwt_entropy_shannon_dec_15
0.9878485361235357 tqwt_stdValue_dec_16 tqwt_entropy_shannon_dec_16
0.8263937663933237 tqwt_stdValue_dec_16 tqwt

0.8098842916950121 tqwt_minValue_dec_3 tqwt_TKEO_mean_dec_2
0.8110805043680349 tqwt_minValue_dec_3 tqwt_TKEO_mean_dec_3
0.8227831582138455 tqwt_minValue_dec_3 tqwt_TKEO_std_dec_1
0.8427206748643835 tqwt_minValue_dec_3 tqwt_TKEO_std_dec_2
0.8337294223052628 tqwt_minValue_dec_3 tqwt_TKEO_std_dec_3
0.8181148466920101 tqwt_minValue_dec_3 tqwt_stdValue_dec_2
0.8427395185742773 tqwt_minValue_dec_3 tqwt_stdValue_dec_3
0.8200158082574605 tqwt_minValue_dec_3 tqwt_stdValue_dec_4
0.957188020984676 tqwt_minValue_dec_3 tqwt_minValue_dec_2
0.8799847448068907 tqwt_minValue_dec_4 tqwt_entropy_shannon_dec_4
0.8389487106174288 tqwt_minValue_dec_4 tqwt_entropy_shannon_dec_5
0.8271682102585804 tqwt_minValue_dec_4 tqwt_stdValue_dec_3
0.9196426485901604 tqwt_minValue_dec_4 tqwt_stdValue_dec_4
0.8900261110277825 tqwt_minValue_dec_4 tqwt_stdValue_dec_5
0.8216455060966553 tqwt_minValue_dec_4 tqwt_stdValue_dec_6
0.8793071706856465 tqwt_minValue_dec_4 tqwt_minValue_dec_2
0.9230248311143774 tqwt_minValue_dec_4 tq

0.8889962747671251 tqwt_minValue_dec_32 tqwt_entropy_shannon_dec_31
0.8730709226965678 tqwt_minValue_dec_32 tqwt_entropy_shannon_dec_32
0.8208123109946331 tqwt_minValue_dec_32 tqwt_entropy_shannon_dec_34
0.8487189250269914 tqwt_minValue_dec_32 tqwt_entropy_shannon_dec_35
0.8557681870415519 tqwt_minValue_dec_32 tqwt_TKEO_mean_dec_31
0.8550758444637636 tqwt_minValue_dec_32 tqwt_TKEO_mean_dec_32
0.9085396295567564 tqwt_minValue_dec_32 tqwt_TKEO_std_dec_32
0.8106669433192539 tqwt_minValue_dec_32 tqwt_TKEO_std_dec_34
0.9142430226834716 tqwt_minValue_dec_32 tqwt_stdValue_dec_31
0.9362575811780277 tqwt_minValue_dec_32 tqwt_stdValue_dec_32
0.8714850545898905 tqwt_minValue_dec_32 tqwt_stdValue_dec_33
0.8960023962189906 tqwt_minValue_dec_32 tqwt_stdValue_dec_34
0.8696451829273634 tqwt_minValue_dec_32 tqwt_stdValue_dec_35
0.8466052996099627 tqwt_minValue_dec_32 tqwt_stdValue_dec_36
0.9249194021835389 tqwt_minValue_dec_32 tqwt_minValue_dec_31
0.8270818454803394 tqwt_minValue_dec_33 tqwt_entropy_sh

0.848629413679655 tqwt_maxValue_dec_10 tqwt_minValue_dec_9
0.9890526807516419 tqwt_maxValue_dec_10 tqwt_minValue_dec_10
0.8908448860472372 tqwt_maxValue_dec_10 tqwt_minValue_dec_11
0.86726946025472 tqwt_maxValue_dec_10 tqwt_maxValue_dec_9
0.8064505176493408 tqwt_maxValue_dec_11 tqwt_TKEO_std_dec_11
0.8458610601398391 tqwt_maxValue_dec_11 tqwt_stdValue_dec_11
0.857199904856544 tqwt_maxValue_dec_11 tqwt_minValue_dec_10
0.976484438670968 tqwt_maxValue_dec_11 tqwt_minValue_dec_11
0.8541473737215809 tqwt_maxValue_dec_11 tqwt_minValue_dec_12
0.8846723188805956 tqwt_maxValue_dec_11 tqwt_maxValue_dec_10
0.805022220354026 tqwt_maxValue_dec_12 tqwt_stdValue_dec_12
0.8456809702594438 tqwt_maxValue_dec_12 tqwt_minValue_dec_11
0.9890066064577201 tqwt_maxValue_dec_12 tqwt_minValue_dec_12
0.8863437381460312 tqwt_maxValue_dec_12 tqwt_minValue_dec_13
0.8414138617567727 tqwt_maxValue_dec_12 tqwt_maxValue_dec_11
0.818903996684998 tqwt_maxValue_dec_13 tqwt_entropy_shannon_dec_13
0.8209635312128479 tqwt_ma

0.8037952077133506 tqwt_maxValue_dec_36 tqwt_minValue_dec_32
0.8501803212963629 tqwt_maxValue_dec_36 tqwt_minValue_dec_34
0.8556413653847 tqwt_maxValue_dec_36 tqwt_minValue_dec_35
0.9440737419359612 tqwt_maxValue_dec_36 tqwt_minValue_dec_36
0.8208464709306927 tqwt_maxValue_dec_36 tqwt_maxValue_dec_32
0.8223781304595662 tqwt_maxValue_dec_36 tqwt_maxValue_dec_33
0.8623626190354285 tqwt_maxValue_dec_36 tqwt_maxValue_dec_34
0.8619555413478485 tqwt_maxValue_dec_36 tqwt_maxValue_dec_35
0.8930160982681679 tqwt_skewnessValue_dec_2 tqwt_skewnessValue_dec_1
0.9209514430748797 tqwt_skewnessValue_dec_4 tqwt_skewnessValue_dec_2
0.8813390759626717 tqwt_skewnessValue_dec_25 tqwt_skewnessValue_dec_24
0.8875534928209362 tqwt_skewnessValue_dec_26 tqwt_skewnessValue_dec_24
0.9977675118690833 tqwt_skewnessValue_dec_26 tqwt_skewnessValue_dec_25
0.9448800754934302 tqwt_skewnessValue_dec_32 tqwt_skewnessValue_dec_31
0.8548334499450313 tqwt_skewnessValue_dec_33 tqwt_skewnessValue_dec_31
0.9211027192506595 tqw

In [14]:
X_female_smote_scaled_correlated = X_female_smote_scaled
X_female_smote_scaled_correlated.drop(labels=corr_features_female, axis=1, inplace=True)


X_female_smote_scaled_correlated.shape

(486, 242)

In [15]:
X_male_smote_scaled_correlated = X_male_smote_scaled
X_male_smote_scaled_correlated.drop(labels=corr_features_male, axis=1, inplace=True)

X_male_smote_scaled_correlated.shape

(642, 261)

# Split para dados de treino e teste

In [16]:
X_train_female, X_test_female, y_train_female, y_test_female=train_test_split(X_female_smote_scaled_correlated,
                                                                              y_female_smote,
                                                                              test_size=0.2, 
                                                                              random_state=42)

In [17]:
X_train_male, X_test_male, y_train_male, y_test_male=train_test_split(X_male_smote_scaled_correlated,
                                                                              y_male_smote,
                                                                              test_size=0.2, 
                                                                              random_state=42)

# PCA

In [18]:
pca = PCA(n_components=0.95).fit(X_train_female.drop(columns='gender'))
pca_train_data = pca.transform(X_train_female.drop(columns='gender'))
pca_test_data = pca.transform(X_test_female.drop(columns='gender'))

X_train_pca_female = pd.DataFrame.from_records(data=pca_train_data)

#reset index to map id and gender to pca data (train)
#X_train_female.reset_index(inplace=True)
#X_train_pca_female['gender'] = X_train_female['gender']

X_test_pca_female = pd.DataFrame.from_records(data=pca_test_data)

#reset index to map id and gender to pca data (test)
#X_test_female.reset_index(inplace=True)
#X_test_pca_female['gender'] = X_test_female['gender']

In [19]:
pca = PCA(n_components=0.95).fit(X_train_male.drop(columns='gender'))
pca_train_data = pca.transform(X_train_male.drop(columns='gender'))
pca_test_data = pca.transform(X_test_male.drop(columns='gender'))

X_train_pca_male = pd.DataFrame.from_records(data=pca_train_data)

#reset index to map id and gender to pca data (train)
#X_train_female.reset_index(inplace=True)
#X_train_pca_male['gender'] = X_train_male['gender']

X_test_pca_male = pd.DataFrame.from_records(data=pca_test_data)

#reset index to map id and gender to pca data (test)
#X_test_female.reset_index(inplace=True)
#X_test_pca_male['gender'] = X_test_male['gender']

In [20]:
def evaluate_classifiers(X_train, y_train, X_test, y_test):
    classifiers = {
        'SVM': {
            'model': SVC(),
            'param_grid': {
                'C': [0.1, 1, 10],
                'kernel': ['linear', 'rbf', 'poly'],
                'gamma': ['scale', 'auto']
            }
        },
        'Random Forest': {
            'model': RandomForestClassifier(),
            'param_grid': {
                'n_estimators': [100, 200, 300],
                'criterion': ['gini', 'entropy'],
                'max_depth': [None, 5, 10]
            }
        },
        'KNN': {
            'model': KNeighborsClassifier(),
            'param_grid': {
                'n_neighbors': [3, 5, 7],
                'weights': ['uniform', 'distance'],
                'p': [1, 2]
            }
        },
        'Naive Bayes': {
            'model': GaussianNB(),
            'param_grid': {
                'priors': [None, [0.1, 0.9], [0.3, 0.7], [0.5, 0.5], [0.7, 0.3], [0.9, 0.1]]

            }
        },
        'Decision Tree': {
            'model': DecisionTreeClassifier(),
            'param_grid': {
                'criterion': ['gini', 'entropy'],
                'max_depth': [None, 5, 10]
            }
        },
        'XGBoost': {
            'model': XGBClassifier(),
            'param_grid': {
                'learning_rate': [0.1, 0.01],
                'max_depth': [3, 5, 7],
                'n_estimators': [100, 200, 300]
            }
        },
        'AdaBoost': {
            'model': AdaBoostClassifier(),
            'param_grid': {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.1, 1, 10]
            }
        }
        
    }

    performance_metrics = {}

    for clf_name, clf_info in classifiers.items():
        clf = clf_info['model']
        param_grid = clf_info['param_grid']

        grid_search = GridSearchCV(clf, param_grid, cv=5)
        grid_search.fit(X_train, y_train)
        best_clf = grid_search.best_estimator_

        y_pred = best_clf.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred)
        performance_metrics[clf_name] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'auc':auc
        }

    return performance_metrics

In [ ]:
results_female = evaluate_classifiers(X_train_female, y_train_female, X_test_female, y_test_female)

for clf_name, metrics in results_female.items():
    print(clf_name + " Performance Metrics:")
    print("Accuracy:", metrics['accuracy'])
    print("Precision:", metrics['precision'])
    print("Recall:", metrics['recall'])
    print("F1 Score:", metrics['f1'])
    print("AUC-ROC:", metrics['auc'])
    print()

In [ ]:
results_male = evaluate_classifiers(X_train_male, y_train_male, X_test_male, y_test_male)

for clf_name, metrics in results_male.items():
    print(clf_name + " Performance Metrics:")
    print("Accuracy:", metrics['accuracy'])
    print("Precision:", metrics['precision'])
    print("Recall:", metrics['recall'])
    print("F1 Score:", metrics['f1'])
    print("AUC-ROC:", metrics['auc'])
    print()